<div style="background-color:rgba(255, 0, 0, 0.5); padding: 10px; border-radius: 5px; width: 95%">

### 🔒 **Empowering LLMs with External Tools**

Large Language Models (LLMs) operate based on static training data and do **not** have real-time awareness or built-in access to current events, live systems, or external APIs.  
Unless explicitly integrated with such tools, they:

- Cannot browse the web  
- Cannot run code  
- Cannot interact with real-time data sources

As a result, when asked questions that require up-to-date knowledge or external action, LLMs may:

- Provide outdated, inaccurate, or fabricated responses, **or**
- Acknowledge their limitations and defer the question
</div>

<div style="background-color:rgba(0, 0, 0, 0.5); padding: 10px; border-radius: 5px; width: 95%">

### 🚀 **Purpose of This Notebook**

This notebook explores how to **extend the capabilities** of LLMs by connecting them to external tools and APIs.  
By integrating real-time services, we can:

- Enable live data access  
- Perform dynamic computations  
- Interact with the environment beyond the training corpus

The goal is to demonstrate practical techniques for transforming LLMs from passive text generators into **interactive, tool-empowered agents**.


</div>


# Start Ollama Server

1. Open a terminal

    ☰ > *Terminal* > *New Terminal*

2. Start the Ollama server 

    ```bash
    ollama serve
    ```

3. Do not close this terminal

4. Open another terminal
5. Pull the model *"qwen3:4b"* (if not already done)

    ```bash
    ollama pull qwen3:4b
    ```
5. Close this terminal



# Define Environment Variables

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434"
LLM_MODEL = "qwen3:4b"
LLM_SEED = 42
LLM_TEMPERATURE = 0.0
TEST_PROMPT0 = "What time is it?"
TEST_PROMPT1 = "What is the price of gold right now?" 
TEST_PROMPT2 = "How many 1 in 111111111111111?"

# Initialize Ollama Chatbot

In [2]:
from langchain_ollama import ChatOllama

# Set up the Ollama chat model with specified LLM model and parameters
llm = ChatOllama(
    base_url=OLLAMA_BASE_URL,
    model=LLM_MODEL,
    temperature=LLM_TEMPERATURE,
    seed=LLM_SEED,
    stream=True
)

# Define Tools

1. **Python Tool:** This tool allows the chatbot to execute Python code, enabling it to perform calculations, data processing, and other tasks that require programming logic.

2. **Commodity Price Tool:** This tool fetches real-time commodity prices from the `API Ninjas` API, allowing the chatbot to provide up-to-date information on various commodities.
    - Go to [``API Ninjas``](https://api-ninjas.com/)
    - Click the "Sign Up" button
    - Create an account
    - Log in
    - Click the "My Account" button
    - Click the "Show API Key" button

    In the next cell, you will need to:
    - Replace the content of the variable `NINJA_API_KEY` with your API key.
    - Run the cell to test the API tool

In [3]:
from langchain_core.tools import tool, Tool
from typing import Annotated, List
import io
import contextlib
import requests

NINJA_API_KEY = "MdaCcVUHseixkwpNGhumWg==9mb676ZThl4du9aP"

@tool
def execute_python(py_code: Annotated[str, "Python code to execute"]) -> str:
    """Executes a Python code and returns its standard output (you have to use the print() function)."""
    output = io.StringIO()
    try:
        with contextlib.redirect_stdout(output):
            exec(py_code, {})
        return output.getvalue().strip() or "Code executed with no output."
    except Exception as e:
        return f"Error: {str(e)}"

# Test the tool
print(execute_python.invoke("print('Hello, World!')"))

@tool
def get_commodity_price(
    commodity: Annotated[str, "The name of the commodity to get the price for"]
) -> str:
    """Get the current price of a commodity using the Ninja API."""
    api_url = 'https://api.api-ninjas.com/v1/commodityprice?name={}'.format(commodity)
    response = requests.get(api_url, headers={'X-Api-Key': NINJA_API_KEY})
    if response.status_code == requests.codes.ok:
        return response.text
    else:
        return f"Error {response.status_code}: {response.text}"

#Test the tool
print(get_commodity_price.invoke("gold"))
    
tools = [execute_python, get_commodity_price]

Hello, World!
{"exchange": "CME", "name": "Gold Futures", "price": 3434.9, "updated": 1749801377}


# Prepare Reason-and-Act (ReAct) Instructions

ReAct (Reason-and-Act) prompting combines:

- **Reasoning:** The model explains its thinking process.
- **Acting:** The model chooses and performs actions (like using a calculator, web search, or database query).

The prompt encourages the model to alternate between these two steps, creating a loop:

→ Think → Act → Observe → Think → Act → … until the task is complete.

**Reference:**

(Yao et al., 2023) Yao, S., Zhao, J., Yu, D., Du, N., Shafran, I., Narasimhan, K., & Cao, Y. (2023, January). ReAct: Synergizing reasoning and acting in language models. In *International Conference on Learning Representations (ICLR)*.]{https://doi.org/10.48550/arXiv.2210.03629}

In [10]:
from inspect import signature

tools_descriptions = "\n- ".join([f"{tool.name}{signature(tool.func)} - {tool.description}" for tool in tools]) 

# Taken from https://smith.langchain.com/hub/hwchase17/react-json
instructions = f"""You are a reasoning and acting assistant. You solve complex tasks by thinking step-by-step and using tools when needed. Follow this format exactly:

Question: <the question you need to answer>

Thought: <your reasoning about what to do next, including whether you need to use a tool or not.>

Action: (What tool to use, if any, and what input to provide. Use the exact JSON format below.)
```json
{{
  "action": <tool name>,
  "action_input": <input for the tool>
}}
```
Observation: <the output from the tool>

... (Repeat Thought → Action → Observation as needed)

Final Answer: <your final answer here>

Important Rules:
- Don't skip steps.
- If an action is needed, use the following format:
  Action: 
  ```json
  {{
    "action": <tool name here>,
    "action_input": <input for the tool here>
  }}
  ```
  Observation: <the output from the tool>
- If no action is needed, give your final answer using the following format:
  Final Answer: <your final answer here>

Available Tools:
- {tools_descriptions}

Think clearly. Be concise. Don't skip steps.
"""

tools_names = [tool.name for tool in tools]
tools_descriptions = "\n".join([f"{tool.name}{signature(tool.func)} - {tool.description}" for tool in tools]) 


instructions =f""""You are a reasoning and acting assistant. Answer the following questions as best you can. You have access to the following tools:

{tools_descriptions}

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: {tools_names}

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```
$JSON_BLOB
```
Observation: the result of the action
... (this Thought/Action/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to always use the exact characters `Final Answer` when responding."""

print(instructions)

"You are a reasoning and acting assistant. Answer the following questions as best you can. You have access to the following tools:

execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Python code and returns its standard output (you have to use the print() function).
get_commodity_price(commodity: Annotated[str, 'The name of the commodity to get the price for']) -> str - Get the current price of a commodity using the Ninja API.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: ['execute_python', 'get_commodity_price']

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

ALWAYS use the following

# Test the Chatbot

In [11]:
from src.Chatbot import Chatbot
from langchain_core.messages import SystemMessage
# Create a chat history with a system message and a human message
chatbot = Chatbot(llm=llm, history=[SystemMessage(content=instructions)])
chatbot.invoke(TEST_PROMPT0)
# chatbot.interact()

================================ System Message ================================

"You are a reasoning and acting assistant. Answer the following questions as best you can. You have access to the following tools:

execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Python code and returns its standard output (you have to use the print() function).
get_commodity_price(commodity: Annotated[str, 'The name of the commodity to get the price for']) -> str - Get the current price of a commodity using the Ninja API.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: ['execute_python', 'get_commodity_price']

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```


AIMessage(content='<think>\nOkay, the user is asking, "What time is it?" Let me think about how to approach this. The tools I have access to are execute_python and get_commodity_price. The get_commodity_price is for getting the price of a commodity, which isn\'t related to the time. The execute_python tool allows me to run Python code. So, maybe I can use that to get the current time.\n\nWait, how do I get the current time in Python? I can use the datetime module. Let me recall. The datetime.datetime.now() function would give the current date and time. Then I can format it to get just the time part. So the code would be something like:\n\nfrom datetime import datetime\ncurrent_time = datetime.now().strftime("%H:%M:%S")\nprint(current_time)\n\nThat should output the current time in hours, minutes, and seconds. Let me check if there\'s any possible error here. The user might be in a different time zone, but since the question is straightforward, I\'ll assume the local time is correct. \n

<div style= "padding: 0.5em;background-color: rgba(255,0,0, 0.5);width: 95%">

### **Problem: LLM Hallucinated the Tool Response**

In this example, the LLM generated the keyword <em>"Observation:"</em> in its response, but we did not actually call the tool at that point. As a result, the LLM fabricated (hallucinated) a tool output instead of providing a real result from the tool.

</div>

# Interrupting the chatbot when generating '*Observations:*'

In [15]:

binded_llm = llm.bind(stop=["Observation:"])
history = [SystemMessage(content=instructions)]
chatbot = Chatbot(llm=binded_llm, history=history)
chatbot.invoke(TEST_PROMPT0)
#chatbot.interact()

================================ System Message ================================

"You are a reasoning and acting assistant. Answer the following questions as best you can. You have access to the following tools:

execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Python code and returns its standard output (you have to use the print() function).
get_commodity_price(commodity: Annotated[str, 'The name of the commodity to get the price for']) -> str - Get the current price of a commodity using the Ninja API.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: ['execute_python', 'get_commodity_price']

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```


KeyboardInterrupt: 

# Create an Agent Class

<div style="background-color:rgba(0, 0, 0, 0); padding: 10px; border: 1px solid rgba(255, 255, 255, 1); border-radius: 5px; width: 95%; margin: 10px auto; text-align: center;">

### 📊 Conversation Flow Diagram

```mermaid
flowchart LR
   State0((START))
   State1(ReAct Instructions)
   State2(Assistant)
   State4(Tools)
   State3(Human)
   State5((END))
   State0 --> State1
   State1 -- SystemMessage --> State2
   State2 -- AiMessage --> State3
   State2 -- AiMessage --> State4
   State4 -- ToolMessage --> State2

   State3 -- HumanMessage --> State2
   State3 -- type `stop` --> State5
```
</div>


In [20]:
%load_ext autoreload
%autoreload 2

from src.Agent import Agent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Testing the Agent

In [21]:
# Create a chat history with a system message and a human message
agent = Agent(llm=llm, tools=tools, history=[SystemMessage(content=instructions)])
agent.invoke(TEST_PROMPT0)
agent.invoke(TEST_PROMPT1)
agent.invoke(TEST_PROMPT2)

================================ System Message ================================

"You are a reasoning and acting assistant. Answer the following questions as best you can. You have access to the following tools:

execute_python(py_code: Annotated[str, 'Python code to execute']) -> str - Executes a Python code and returns its standard output (you have to use the print() function).
get_commodity_price(commodity: Annotated[str, 'The name of the commodity to get the price for']) -> str - Get the current price of a commodity using the Ninja API.

The way you use the tools is by specifying a json blob.
Specifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are: ['execute_python', 'get_commodity_price']

The $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:

```


### To go further

- ReAct limitations: the LLM must wait for the tool to finish before continuing its reasoning (it is slow).  
  To overcome this limitation, you can use the [LLMCompiler](https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/rewoo/rewoo.ipynb). The LLMCompiler allows the LLM to generate a plan of actions, which can then be executed in parallel. Once all actions are completed, the LLM can then process the results and provide a final response or trigger a another plan.
  It has 3 main components:
    - Planner: stream a [Directed Acyclic Graphs (DAGs)]{https://microsoft.github.io/promptflow/how-to-guides/develop-a-dag-flow/index.html} of function calls.
    - Task Fetching Unit: schedules and executes the tasks as soon as they are executable
    - Joiner: Responds to the user or triggers a second plan


 Explore Model Context Protocol (MCP) which allows you to make your tools and APIs publicly available to the LLMs.
